In [1]:
# Reading and data and display sample

import pandas as pd
import os
import torch
from transformers import BertTokenizer, BertModel
from datasets import load_dataset, load_metric
import time
import json
from pathlib import PurePath
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f'device is {device}')


device is cpu


In [2]:
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
file_path = '/Users/nik/Documents/Codes/random_stuff/Final_end/data/nikhil_onlytextpytorch.json'

# path = os.path.join(os.getcwd(),file_path)

print(path)

train = pd.read_json(file_path, orient='index')


/Users/nik/Documents/Codes/random_stuff/Final_end/data_cleanup/data/nikhil_onlytextpytorch.json


In [6]:

train

,answer,question,context,source
"What values are specified to replaceNaN, positive infinity, and negative infinity values in input?","bynan,posinf, andneginf","What values are specified to replaceNaN, posit...","ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
"By default,NaN is replaced with what value?",zero,"By default,NaN is replaced with what value?","ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
What values are used to replace negative infinity values in input?,"bynan,posinf, andneginf",What values are used to replace negative infin...,"ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
What is the default value for negative infinity?,the least finite value,What is the default value for negative infinity?,"ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
What is the value to replace positive infinity values with?,posinf,What is the value to replace positive infinity...,"nan(Number,optional) – the value to replaceNaN...",https://pytorch.org/docs/stable/generated/torc...
...,...,...,...,...
What is the tensor of shape where*is zero or more batch dimensions?,A(Tensor),What is the tensor of shape where*is zero or m...,torch.linalg.inv()computes the inverse of a sq...,https://pytorch.org/docs/stable/generated/torc...
What is the tolerance value to determine when is a singular value zero?,rcond,What is the tolerance value to determine when ...,torch.linalg.lstsq()computesA.pinv() @Bwith a\...,https://pytorch.org/docs/stable/generated/torc...
What is the default value of atorch.Tensor?,1e-15,What is the default value of atorch.Tensor?,See also torch.linalg.inv()computes the invers...,https://pytorch.org/docs/stable/generated/torc...
What computes the inverse of a square matrix?,torch.linalg.inv(),What computes the inverse of a square matrix?,See also torch.linalg.inv()computes the invers...,https://pytorch.org/docs/stable/generated/torc...


In [7]:

train_data = train.reset_index()[['question', 'answer', 'context']]

In [8]:
train_data = train_data.reset_index().rename(columns={'index': 'id'})
display(train_data)

,id,question,answer,context
0,0,"What values are specified to replaceNaN, posit...","bynan,posinf, andneginf","ReplacesNaN, positive infinity, and negative i..."
1,1,"By default,NaN is replaced with what value?",zero,"ReplacesNaN, positive infinity, and negative i..."
2,2,What values are used to replace negative infin...,"bynan,posinf, andneginf","ReplacesNaN, positive infinity, and negative i..."
3,3,What is the default value for negative infinity?,the least finite value,"ReplacesNaN, positive infinity, and negative i..."
4,4,What is the value to replace positive infinity...,posinf,"nan(Number,optional) – the value to replaceNaN..."
...,...,...,...,...
998,998,What is the tensor of shape where*is zero or m...,A(Tensor),torch.linalg.inv()computes the inverse of a sq...
999,999,What is the tolerance value to determine when ...,rcond,torch.linalg.lstsq()computesA.pinv() @Bwith a\...
1000,1000,What is the default value of atorch.Tensor?,1e-15,See also torch.linalg.inv()computes the invers...
1001,1001,What computes the inverse of a square matrix?,torch.linalg.inv(),See also torch.linalg.inv()computes the invers...


In [9]:
df_groups = train_data.groupby('context')['id'].apply(list).reset_index(name = 'groups')

In [10]:
df_groups = df_groups.reset_index().rename(columns={'index': 'group_id'})

In [11]:
df_groups

,group_id,context,groups
0,0,A wrapper around C++torch::jit::Module. Fu...,"[590, 595, 596]"
1,1,Alias for torch.asinh(). Returns a new ten...,"[256, 260, 271, 287, 288, 289, 290, 291, 293, ..."
2,2,Alias fortorch.asinh(). Returns a new tens...,"[679, 681, 683, 685]"
3,3,Alias fortorch.asinh(). Returns a new tens...,[673]
4,4,Alias of torch.outer(). Computes the dot p...,"[876, 880]"
...,...,...,...
343,343,window_length(int) – the size of returned wind...,"[54, 95, 96, 100, 101, 102]"
344,344,– Number – sum(abs(x)**ord)**(1./ord) The vect...,[437]
345,345,– sum(abs(x)**ord)**(1./ord) The vector norm c...,[881]
346,346,‘nuc’ nuclear norm – Number – sum(abs(x)**ord)...,"[433, 434]"


In [18]:
df_merged = pd.merge(train_data, df_groups).reset_index(drop=True)

In [19]:
df_merged = df_merged[['question', 'answer', 'group_id']]

In [20]:
df_merged

,question,answer,group_id
0,"What values are specified to replaceNaN, posit...","bynan,posinf, andneginf",141
1,"By default,NaN is replaced with what value?",zero,141
2,What values are used to replace negative infin...,"bynan,posinf, andneginf",141
3,What is the default value for negative infinity?,the least finite value,141
4,What is the value to replace positive infinity...,posinf,295
...,...,...,...
998,What are the singular values that are below th...,zero,225
999,What function synchronizes a CUDA input with t...,usestorch.linalg.svd()ifhermitian= False,225
1000,Why is it always preferable to uselstsq()?,faster and more numerically stable,243
1001,What is the default value of atorch.Tensor?,1e-15,193


In [21]:
df_merged.to_csv('qna_gid.csv')

In [24]:
display(df_groups.iloc[141])


group_id                                                  141
context     ReplacesNaN, positive infinity, and negative i...
groups                                           [0, 1, 2, 3]
Name: 141, dtype: object

In [16]:
df_groups.to_csv('groups_master.csv')

In [59]:
train_data = train_data.reset_index().rename(columns={'index': 'id'})
# display(train_data)

In [142]:
display(train_data)

,id,question,answer,context
0,0,"What values are specified to replaceNaN, posit...","bynan,posinf, andneginf","ReplacesNaN, positive infinity, and negative i..."
1,1,"By default,NaN is replaced with what value?",zero,"ReplacesNaN, positive infinity, and negative i..."
2,2,What values are used to replace negative infin...,"bynan,posinf, andneginf","ReplacesNaN, positive infinity, and negative i..."
3,3,What is the default value for negative infinity?,the least finite value,"ReplacesNaN, positive infinity, and negative i..."
4,4,What is the value to replace positive infinity...,posinf,"nan(Number,optional) – the value to replaceNaN..."
...,...,...,...,...
998,998,What is the tensor of shape where*is zero or m...,A(Tensor),torch.linalg.inv()computes the inverse of a sq...
999,999,What is the tolerance value to determine when ...,rcond,torch.linalg.lstsq()computesA.pinv() @Bwith a\...
1000,1000,What is the default value of atorch.Tensor?,1e-15,See also torch.linalg.inv()computes the invers...
1001,1001,What computes the inverse of a square matrix?,torch.linalg.inv(),See also torch.linalg.inv()computes the invers...


In [7]:
from tqdm import tqdm

def get_bert_tokens(text, max_length=512):
    tk = tokenizer(
            text[3],
            max_length=max_length,
            padding='max_length',
            return_attention_mask=True,
            add_special_tokens=True,
            truncation=True,
            return_tensors="pt"
        )
    model_op = model(**tk)
    return model_op

In [8]:
records = set([i['context'].strip() for i in train_data.to_dict('records')])

In [16]:
100%100

0

In [9]:
import gc
new_tensor_dict = {}

for index, i in tqdm(enumerate(records)):
    new_tensor_dict[index] = {'text':i, 'model_op': get_bert_tokens(i)}
    if index%100==0:
        torch.save(new_tensor_dict, f'file_{index}.pt')
        new_tensor_dict = {}
        gc.collect()

348it [08:09,  1.41s/it]


In [123]:
records = set([i['context'].strip() for i in train_data.to_dict('records')])

print(len(records))


print(list(records)[8])
# new_tensor_dict = {}

# for i in tqdm(records):
#     new_tensor_dict[] = {'model_op': inference(i), 'top_similar' : [], 'least_similar':[]}

348
‘nuc’ nuclear norm – Number – sum(abs(x)**ord)**(1./ord) The vector norm can be calculated across any number of dimensions.
The corresponding dimensions of inputare flattened into
one dimension, and the norm is calculated on the flattened
dimension. Frobenius norm produces the same result asp=2in all cases
except whendimis a list of three or more dims, in which
case Frobenius norm throws an error. Nuclear norm can only be calculated across exactly two dimensions.


In [129]:
lengths = [len(i) for i in records]


In [10]:
index = 400
torch.save(new_tensor_dict, f'file_{index}.pt')

In [132]:
sum(lengths)/len(lengths)

1097.83908045977

In [139]:
list(zip(lengths, records))[1]

(2033,
 'torch.acosh().   Adds the scalar otherto each element of the inputinputand returns a new resulting tensor.   Performs the element-wise division of tensor1 by tensor2, multiply the result by the scalarvalueand add it toinput.   Performs the element-wise multiplication of tensor1 by tensor2, multiply the result by the scalarvalueand add it toinput.   Computes the element-wise angle (in radians) of the giveninputtensor.   Returns a new tensor with the arcsine  of the elements of input.   Alias for torch.asin().   Returns a new tensor with the inverse hyperbolic sine of the elements of input.   Alias for torch.asinh().   Returns a new tensor with the arctangent  of the elements of input.   Alias for torch.atan().   Returns a new tensor with the inverse hyperbolic tangent of the elements of input.   Alias for torch.atanh().   Element-wise arctangent of inputi/otheri\\text{input}_{i} / \\text{other}_{i}inputi\u200b/otheri\u200bwith consideration of the quadrant.   Computes the bitwi

In [ ]:
list(zip(lengths, records))[1]

In [7]:
dict2 = torch.load('tensor_dict.pt')

NameError: name 'torch' is not defined

In [153]:
# count of sentences above and below 1k

sum([1 for i in records if len(i) < 512])

139

In [11]:
dict2.update(tensor_dict)

In [12]:
torch.save(dict2, 'tensor_dict1.pt')

In [ ]:
def save_encodings(id, text):
    dataset = TextTokenizer(train, tokenizer, column_name='context')
    for text in dataset:
        model(**text)

In [11]:
torch.save(tensor_dict, 'tensor_dict.pt')

# Get cosine similarity between vectors

In [ ]:
def get_cosine_similarity():
    input1 = torch.randn(100, 128)
    input2 = torch.randn(100, 128)
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    output = cos(input1, input2)

In [2]:
import faiss
import numpy as np


In [6]:

dataSetI = [.1, .2, .3]
dataSetII = [.4, .5, .6]
#dataSetII = [.1, .2, .3]

x = np.array([dataSetI]).astype(np.float32)
q = np.array([dataSetII]).astype(np.float32)
index = faiss.index_factory(3, "Flat", faiss.METRIC_INNER_PRODUCT)
index.ntotal
faiss.normalize_L2(x)
index.add(x)
faiss.normalize_L2(q)
distance, index = index.search(q, 5)

# norm_distance = 25 * (4 - distance)
print(f'Distance by FAISS:{distance}, Index:{index}')



Distance by FAISS:[[ 9.7463179e-01 -3.4028235e+38 -3.4028235e+38 -3.4028235e+38
  -3.4028235e+38]], Index:[[ 0 -1 -1 -1 -1]]


In [8]:
import torch
dict2 = torch.load('tensor_dict.pt')

In [19]:
for i in dict2.values():
    print(i['model_op']['pooler_output'].shape)
    break

torch.Size([1, 768])


In [21]:
a = torch.randn([1, 768])
b = torch.randn([1, 768])

In [25]:
torch.cat((a, b), 0).shape

torch.Size([2, 768])

In [ ]:
# Get list of tensors from dict

In [39]:
final_op.shape

torch.Size([2, 768])

In [41]:
a = [i['model_op']['pooler_output'] for i in dict1.values()]

In [42]:
len(a)

347

In [27]:
dict1 = torch.load('file_100.pt')

In [28]:
dict2 = torch.load('file_200.pt')

In [29]:
dict1.update(dict2)

In [31]:
dict3 = torch.load('file_300.pt')

In [32]:
dict4 = torch.load('file_400.pt')

In [30]:
dict1.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200])

In [33]:
dict1.update(dict3)

In [34]:
dict1.update(dict4)

In [39]:
final_op = [i['model_op'] for i in dict1.values()]

In [43]:
final_op = torch.cat(a, 0)

In [45]:
final_op.shape

torch.Size([347, 768])

In [47]:
shape = final_op.shape[1]

In [ ]:
d = 

In [48]:
import faiss

index = faiss.IndexFlatL2(shape)
print(index.is_trained)
index.add(final_op.cpu().detach().numpy())
print(index.ntotal)

In [68]:
token_text = r"""Concatenates the given sequence ofseqtensors in the given dimension.   Splits a tensor into a specific number of chunks.   Splitsinput, a tensor with three or more dimensions, into multiple tensors depthwise according toindices_or_sections.   Creates a new tensor by horizontally stacking the tensors in tensors.   Stack tensors in sequence depthwise (along third axis).   Gathers values along an axis specified by dim.   Splitsinput, a tensor with one or more dimensions, into multiple tensors horizontally according toindices_or_sections.   Stack tensors in sequence horizontally (column wise).   Returns a new tensor which indexes theinputtensor along dimensiondimusing the entries inindexwhich is a LongTensor.   Returns a new 1-D tensor which indexes theinputtensor according to the boolean maskmaskwhich is a BoolTensor.   Moves the dimension(s) of input at the position(s) insourceto the position(s) indestination.   Alias for torch.movedim().   Returns a new tensor that is a narrowed version of inputtensor.      Returns a tensor with the same data and number of elements asinput, but with the specified shape.   Alias of torch.vstack().   Out-of-place version of torch.Tensor.scatter_()   Out-of-place version of torch.Tensor.scatter_add_()   Splits the tensor into chunks.   Returns a tensor with all the dimensions of inputof size1removed.   Concatenates a sequence of tensors along a new dimension.   Alias for torch.transpose().   Alias for torch.transpose().   Expectsinputto be <= 2-D tensor and transposes dimensions 0 and 1.   Returns a new tensor with the elements of input at the given indices.   Selects values frominput at the 1-dimensional indices fromindicesalong the givendim.   Splits a tensor into multiple sub-tensors, all of which are views of input, along dimensiondimaccording to the indices or number of sections specified byindices_or_sections.   Constructs a tensor by repeating the elements of input.   Returns a tensor that is a transposed version of input.   Removes a tensor dimension."""

max_length=512

tokens = tokenizer(
            token_text,
            max_length=max_length,
            padding='max_length',
            return_attention_mask=True,
            add_special_tokens=True,
            truncation=True,
            return_tensors="pt")

In [69]:
token_index = model(**tokens)['pooler_output']

In [70]:
D, I = index.search(token_index.cpu().detach().numpy(), 100)

In [71]:
print(D, I)

[[126.97375 180.0263  180.0263  180.0263  180.0263  180.0263  180.0263
  180.0263  180.0263  180.0263  180.0263  180.0263  180.0263  180.0263
  180.0263  180.0263  180.0263  180.0263  180.0263  180.0263  180.0263
  180.0263  180.0263  180.0263  180.0263  180.0263  180.0263  180.0263
  180.0263  180.0263  180.0263  235.4931  235.4931  253.60294 253.60294
  253.60294 254.90831 254.90831 254.90831 254.90831 254.90831 254.90831
  254.90831 254.90831 257.8427  257.8427  257.8427  257.8427  257.8427
  257.8427  257.8427  257.8427  257.8427  257.8427  257.8427  257.8427
  257.8427  257.8427  257.8427  257.8427  261.40375 261.40375 261.40375
  261.40375 261.40375 261.40375 261.40375 261.40375 261.40375 261.40375
  261.40375 261.40375 261.40375 261.40375 261.40375 261.40375 261.40375
  261.40375 261.40375 261.40375 261.40375 261.40375 261.40375 261.40375
  261.40375 261.40375 261.40375 261.40375 261.40375 261.40375 261.40375
  261.40375 261.40375 261.40375 261.40375 261.40375 261.40375 261.4037

In [67]:
list(records)[1]

'Returns True if theinputis a single element tensor which is not equal to zero after type conversions.   Sets the default floating point dtype tod.   Get the current default floating pointtorch.dtype.   Sets the defaulttorch.Tensortype to floating point tensor typet.   Returns the total number of elements in theinputtensor.   Set options for printing.   Disables denormal floating numbers on CPU. Note Random sampling creation ops are listed underRandom samplingand\ninclude:torch.rand()torch.rand_like()torch.randn()torch.randn_like()torch.randint()torch.randint_like()torch.randperm()You may also usetorch.empty()with theIn-place random samplingmethods to createtorch.Tensors with values sampled from a broader\nrange of distributions.   Constructs a tensor withdata.   Constructs asparse tensor in COO(rdinate) formatwith specified values at the givenindices.   Convert the data into atorch.Tensor.   Create a view of an existingtorch.Tensorinputwith specifiedsize,strideandstorage_offset.   Cre

In [76]:
nlist = 5
quantizer = faiss.IndexFlatL2(shape)
index1 = faiss.IndexIVFFlat(quantizer, shape, nlist)

In [77]:
index1.is_trained

False

In [78]:
index1.train(final_op.cpu().detach().numpy())

In [79]:
index1.is_trained

True

In [83]:
index1.add(final_op.cpu().detach().numpy())

In [85]:
D, I = index1.search(token_index.cpu().detach().numpy(), 100)

In [86]:
print(D, I)

[[1.2697375e+02 1.8002631e+02 1.8002631e+02 1.8002631e+02 1.8002631e+02
  1.8002631e+02 1.8002631e+02 1.8002631e+02 1.8002631e+02 1.8002631e+02
  1.8002631e+02 1.8002631e+02 1.8002631e+02 1.8002631e+02 1.8002631e+02
  1.8002631e+02 1.8002631e+02 1.8002631e+02 1.8002631e+02 1.8002631e+02
  1.8002631e+02 1.8002631e+02 1.8002631e+02 1.8002631e+02 1.8002631e+02
  1.8002631e+02 1.8002631e+02 1.8002631e+02 1.8002631e+02 1.8002631e+02
  1.8002631e+02 3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38
  3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38
  3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38
  3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38
  3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38
  3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38
  3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38
  3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028

In [84]:
index1.ntotal

347

In [107]:
b = list(records)[0]

In [92]:
a = list(records)[:10]

In [95]:
a.append(b)

In [100]:
token_text = b
max_length=512

tokens = tokenizer(
            a,
            max_length=max_length,
            padding='max_length',
            return_attention_mask=True,
            add_special_tokens=True,
            truncation=True,
            return_tensors="pt")

In [105]:
import faiss

index = faiss.IndexFlatL2(shape)
print(index.is_trained)
index.add(token_index.cpu().detach().numpy())
print(index.ntotal)

True
11


In [102]:
token_index = model(**tokens)['pooler_output']

In [110]:
D, I = index.search(b.cpu().detach().numpy(), 5)

In [111]:
print(D, I)

[[6.4574947e-12 6.4574947e-12 1.6628124e+01 2.1520056e+01 2.3134634e+01]] [[10  0  1  9  4]]


In [108]:
tokens = tokenizer(
            b,
            max_length=max_length,
            padding='max_length',
            return_attention_mask=True,
            add_special_tokens=True,
            truncation=True,
            return_tensors="pt")

b = model(**tokens)['pooler_output']

In [109]:
b.shape

torch.Size([1, 768])

In [116]:
# We will use metric inner product as faiss for similarity matching

index = faiss.index_factory(768, "Flat", faiss.METRIC_INNER_PRODUCT)
index.ntotal
faiss.normalize_L2(token_index.cpu().detach().numpy())
index.add(token_index.cpu().detach().numpy())
faiss.normalize_L2(b.cpu().detach().numpy())
distance, index = index.search(b.cpu().detach().numpy(), 10)

# norm_distance = 25 * (4 - distance)
print(f'Distance by FAISS:{distance}, Index:{index}')


Distance by FAISS:[[1.0000002  1.0000002  0.9655337  0.9551681  0.95176923 0.9280489
  0.9217062  0.88812137 0.8688997  0.7612742 ]], Index:[[10  0  1  9  4  7  6  8  2  3]]


In [ ]:

def clean_unnecessary_spaces(out_string):
    if not isinstance(out_string, str):
        warnings.warn(f">>> {out_string} <<< is not a string.")
        out_string = str(out_string)
    out_string = (
        out_string.replace(" .", ".")
        .replace(" ?", "?")
        .replace(" !", "!")
        .replace(" ,", ",")
        .replace(" ' ", "'")
        .replace(" n't", "n't")
        .replace(" 'm", "'m")
        .replace(" 's", "'s")
        .replace(" 've", "'ve")
        .replace(" 're", "'re")
    )

In [12]:
import glob
file_list = (glob.glob("data/*.json"))

In [13]:
len(file_list)

8

## Creating Commmon data model


In [43]:
import os
import pandas as pd


file_path1 = file_list[0]
file_path2 = file_list[1]
file_path3 = file_list[2]
file_path4 = file_list[3]
file_path5 = file_list[4]
file_path6 = file_list[5]
file_path7 = file_list[6]
file_path8 = file_list[7]
# file_path9 = file_list[8]


path1 = os.path.join(os.getcwd(),file_path1)
path2 = os.path.join(os.getcwd(),file_path2)
path3 = os.path.join(os.getcwd(),file_path3)
path4 = os.path.join(os.getcwd(),file_path4)
path5 = os.path.join(os.getcwd(),file_path5)
path6 = os.path.join(os.getcwd(),file_path6)
path7 = os.path.join(os.getcwd(),file_path7)
path8 = os.path.join(os.getcwd(),file_path8)
# path9 = os.path.join(os.getcwd(),file_path9)

print(path5)
train1 = pd.read_json(path1)
train2 = pd.read_json(path2, orient='index')
train3 = pd.read_json(path3)
train4 = pd.read_json(path4)
train5 = pd.read_json(path5)
train6 = pd.read_json(path6)
train7 = pd.read_json(path7)
train8 = pd.read_json(path8)
# train9 = pd.read_json(path9, orient='index')

/Users/nik/Documents/Codes/random_stuff/TSAI_END2_Phase1/data/capstone-1-100.json


In [50]:
train6

,data
0,"{'x': 'What are arguments to embedding layer',..."
1,{'x': 'does embedding layer need size of the ...
2,{'x': 'does embedding layer need embedding di...
3,{'x': 'Does the embedding layer require an emb...
4,{'x': 'Is embedding dimension required for the...
...,...
95,{'x': ' ptorch geometric has multiple gpu sup...
96,{'x': 'Is ptorch geometric compatible with man...
97,{'x': 'are multiple gpus compatible with pytor...
98,{'x': 'multiple gpus compatible with pytorch g...


In [54]:
col2_expanded = train6.data.apply(lambda x:pd.Series(x))
col2_expanded.columns = ['{}.{}'.format('data',i) for i in col2_expanded]
col2_expanded

,data.x,data.y,data.z
0,What are arguments to embedding layer,The Embedding layer that is used here is param...,The Embedding layer that is used here is para...
1,does embedding layer need size of the vocabul...,The Embedding layer that is used here is param...,The Embedding layer that is used here is para...
2,does embedding layer need embedding dimension?,The Embedding layer that is used here is param...,The Embedding layer that is used here is para...
3,Does the embedding layer require an embedding ...,The Embedding layer that is used here is param...,The Embedding layer that is used here is para...
4,Is embedding dimension required for the embedd...,The Embedding layer that is used here is param...,The Embedding layer that is used here is para...
...,...,...,...
95,ptorch geometric has multiple gpu support?,PyTorch Geometric (PyG) is a geometric deep l...,PyTorch Geometric (PyG) is a geometric deep le...
96,Is ptorch geometric compatible with many GPUs?,PyTorch Geometric (PyG) is a geometric deep l...,PyTorch Geometric (PyG) is a geometric deep le...
97,are multiple gpus compatible with pytorch geom...,PyTorch Geometric (PyG) is a geometric deep l...,PyTorch Geometric (PyG) is a geometric deep le...
98,multiple gpus compatible with pytorch geometric?,PyTorch Geometric (PyG) is a geometric deep l...,PyTorch Geometric (PyG) is a geometric deep le...


In [56]:
train5

,data
0,"{'x': 'What is PyTorch', 'y': 'PyTorch torch ..."
1,{'x': 'Which package manager is recommended fo...
2,"{'x': 'Why you need CUDA toolkit?', 'y': 'CUDA..."
3,"{'x': 'Why you need CUDA toolkit?', 'y': 'CUDA..."
4,"{'x': 'How pytorch replaces numpy?', 'z': 'bas..."
...,...
95,{'x': 'what is use of backward() functntion of...
96,{'x': 'which function is used to propagate los...
97,{'x': 'how to propagate loss backwards in pyto...
98,{'x': 'which function is used to update value ...


In [57]:


col3_expanded = train5.data.apply(lambda x:pd.Series(x))
col3_expanded.columns = ['{}.{}'.format('data',i) for i in col3_expanded]
col3_expanded

,data.x,data.y,data.z,data.video_id
0,What is PyTorch,PyTorch torch is one of the most popular mach...,hi everybody welcome to a new tutorial series...,EMXfZB8FVUA
1,Which package manager is recommended for PyTorch?,Anaconda package manager is recommended to ins...,hi everybody welcome to a new tutorial series...,EMXfZB8FVUA
2,Why you need CUDA toolkit?,CUDA toolkit is a development environment for ...,if you haven't installed anaconda yet and don...,EMXfZB8FVUA
3,Why you need CUDA toolkit?,CUDA toolkit is a development environment for ...,if you haven't installed anaconda yet and don...,EMXfZB8FVUA
4,How pytorch replaces numpy?,unlike numpy PyTorch uses power of GPUs and ot...,basically enabling researchers to be as expres...,NaN
...,...,...,...,...
95,what is use of backward() functntion of loss o...,iteratively propagates the loss backward throu...,Let’s take a look at how this gradient­steppin...,NaN
96,which function is used to propagate loss backw...,backward() iteratively propagates the loss bac...,Let’s take a look at how this gradient­steppin...,NaN
97,how to propagate loss backwards in pytorch?,backward() iteratively propagates the loss bac...,Let’s take a look at how this gradient­steppin...,NaN
98,which function is used to update value of weights,the optimizer (opt) instructs the parameters ...,Let’s take a look at how this gradient­steppin...,NaN


In [134]:

df_56 = pd.concat([col3_expanded, col2_expanded]).drop(columns='data.video_id').rename(columns={
    'data.x': 'question',
    'data.y': 'answer',
    'data.z': 'context'
})


In [136]:
df_56

,question,answer,context
0,What is PyTorch,PyTorch torch is one of the most popular mach...,hi everybody welcome to a new tutorial series...
1,Which package manager is recommended for PyTorch?,Anaconda package manager is recommended to ins...,hi everybody welcome to a new tutorial series...
2,Why you need CUDA toolkit?,CUDA toolkit is a development environment for ...,if you haven't installed anaconda yet and don...
3,Why you need CUDA toolkit?,CUDA toolkit is a development environment for ...,if you haven't installed anaconda yet and don...
4,How pytorch replaces numpy?,unlike numpy PyTorch uses power of GPUs and ot...,basically enabling researchers to be as expres...
...,...,...,...
95,ptorch geometric has multiple gpu support?,PyTorch Geometric (PyG) is a geometric deep l...,PyTorch Geometric (PyG) is a geometric deep le...
96,Is ptorch geometric compatible with many GPUs?,PyTorch Geometric (PyG) is a geometric deep l...,PyTorch Geometric (PyG) is a geometric deep le...
97,are multiple gpus compatible with pytorch geom...,PyTorch Geometric (PyG) is a geometric deep l...,PyTorch Geometric (PyG) is a geometric deep le...
98,multiple gpus compatible with pytorch geometric?,PyTorch Geometric (PyG) is a geometric deep l...,PyTorch Geometric (PyG) is a geometric deep le...


In [91]:
ri = str(random.randint(1,100)) 
col4_expanded = train1.data.apply(lambda x:pd.Series(x))
col4_expanded.columns = ['{}.{}'.format('data',i) for i in col4_expanded]
col4_expanded

,data.X,data.Z,data.Y,data.Y,data.Z,data.X
0,calculating loss for entire batch using nlllos...,"Defake:loss = criterion_test(dec_outs.view(-1,...",I believe you should use criterion test(dec ou...,NaN,NaN,NaN
1,masked fill operates weirdly,"From the error mesage, it is a size issue on t...",The error message indicates that there is a si...,NaN,NaN,NaN
2,creating custom nn module for softmargin softmax,"Prefer not to use for loops, try to vectorize ...","Avoid using for loops whenever feasible, and a...",NaN,NaN,NaN
3,gru autoencoder is not working,I solve this problem.Target tensor was wrong.,Target tensor was wrong,NaN,NaN,NaN
4,issue with multiple gpu loss convergence,I solved my issue. Since batch wasnt my first ...,"mention dim=1 in the data parallel, that is t...",NaN,NaN,NaN
...,...,...,...,...,...,...
548,current status of automatic quantization support,"Yes, graph mode quantization will be ready for...","Yes, graph mode quantization will be ready for...",NaN,NaN,NaN
549,check if grad is enabled,You can check it by accessing torch.Tensor's r...,You may check it out by going to torch. The re...,NaN,NaN,NaN
550,non gradient trackable convolutions,"Yes, that's possible.You could either use the ...",You could either use the functional API with a...,NaN,NaN,NaN
551,best way to encode latent variable with cnn,"I think you might need a flatten in 1, too,We ...","I think you might need a flatten in 1, too.",NaN,NaN,NaN


In [137]:
df_1 = col4_expanded.drop(columns=['data.Z','data.X','data.Y']).rename(columns={
    'data.X ': 'question',
    'data.Y ': 'answer',
    'data.Z ': 'context'
})

In [138]:
df_1

,question,context,answer
0,calculating loss for entire batch using nlllos...,"Defake:loss = criterion_test(dec_outs.view(-1,...",I believe you should use criterion test(dec ou...
1,masked fill operates weirdly,"From the error mesage, it is a size issue on t...",The error message indicates that there is a si...
2,creating custom nn module for softmargin softmax,"Prefer not to use for loops, try to vectorize ...","Avoid using for loops whenever feasible, and a..."
3,gru autoencoder is not working,I solve this problem.Target tensor was wrong.,Target tensor was wrong
4,issue with multiple gpu loss convergence,I solved my issue. Since batch wasnt my first ...,"mention dim=1 in the data parallel, that is t..."
...,...,...,...
548,current status of automatic quantization support,"Yes, graph mode quantization will be ready for...","Yes, graph mode quantization will be ready for..."
549,check if grad is enabled,You can check it by accessing torch.Tensor's r...,You may check it out by going to torch. The re...
550,non gradient trackable convolutions,"Yes, that's possible.You could either use the ...",You could either use the functional API with a...
551,best way to encode latent variable with cnn,"I think you might need a flatten in 1, too,We ...","I think you might need a flatten in 1, too."


In [77]:
import random
random.randint(1,)

6

In [118]:

df_2 = train2.reset_index().drop(columns=['source', 'index'])

In [139]:
df_2

,answer,question,context
0,"bynan,posinf, andneginf","What values are specified to replaceNaN, posit...","ReplacesNaN, positive infinity, and negative i..."
1,zero,"By default,NaN is replaced with what value?","ReplacesNaN, positive infinity, and negative i..."
2,"bynan,posinf, andneginf",What values are used to replace negative infin...,"ReplacesNaN, positive infinity, and negative i..."
3,the least finite value,What is the default value for negative infinity?,"ReplacesNaN, positive infinity, and negative i..."
4,posinf,What is the value to replace positive infinity...,"nan(Number,optional) – the value to replaceNaN..."
...,...,...,...
998,A(Tensor),What is the tensor of shape where*is zero or m...,torch.linalg.inv()computes the inverse of a sq...
999,rcond,What is the tolerance value to determine when ...,torch.linalg.lstsq()computesA.pinv() @Bwith a\...
1000,1e-15,What is the default value of atorch.Tensor?,See also torch.linalg.inv()computes the invers...
1001,torch.linalg.inv(),What computes the inverse of a square matrix?,See also torch.linalg.inv()computes the invers...


In [140]:
df_47 = pd.concat([train7, train4]).drop(columns='null').rename(columns={
    'x': 'question',
    'y': 'answer',
    'z': 'context'
})

In [141]:
df_47


,question,context,answer
0,NaN,NaN,NaN
1,Support multiple simultaneous LR schedulers,A solution was implemented in #26423. Closing ...,This issue has been fixed
2,Want RTX 2080ti Support!!! RuntimeError: cubla...,We'll supply PyTorch binaries w/ CUDA 10 in th...,You need to build from source
3,Crash when reading pandas parquet file after i...,uninstall the pyarrow installed by pip and the...,You need to uninstall the pyarrow installed by...
4,"RuntimeError: Only tuples, lists and Variables...",Solved.Just change the output of your model fr...,You need to change the output of your model fr...
...,...,...,...
1991,torch.autograd.jacobian returns tensors with a...,This is not a bug. It happens because in your ...,This is not a bug. It happens because in your ...
1992,[Bug] Sometimes gradient doesn't back-propagat...,I don't think this is a bug. He is checking th...,I don't think this is a bug. He is checking th...
1993,pytorch 0.4.0 always allocates memory on GPU:0...,I believe this has been fixed in https://githu...,I believe this has been fixed in https://githu...
1994,roi_crop (from Detectron.pytorch) building con...,@phalexo You could try out to install `pytorch...,@phalexo You could try out to install `pytorch...


In [148]:
train8

col4_expanded = train8.data.apply(lambda x:pd.Series(x))
col4_expanded.columns = ['{}.{}'.format('data',i) for i in col4_expanded]
col4_expanded

df_8 = col4_expanded.rename(columns={
    'data.x': 'question',
    'data.y': 'answer',
    'data.z': 'context'
})

df_8

,question,answer,context
0,What does torchvision.transforms.CenterCrop do?,Crops the given image at the center,torchvision.transforms.CenterCrop(size)[SOURC...
1,What is the purpose of torchvision.transforms....,Crops the given image at the center,torchvision.transforms.CenterCrop(size)[SOURC...
2,What is torchvision.transforms.CenterCrop used...,Crops the given image at the center,torchvision.transforms.CenterCrop(size)[SOURC...
3,is center cropping available?,torchvision.transforms.CenterCrop Crops the g...,torchvision.transforms.CenterCrop(size)[SOURC...
4,is there center cropping available in torchvis...,torchvision.transforms.CenterCrop Crops the g...,torchvision.transforms.CenterCrop(size)[SOURC...
...,...,...,...
96,Why isn't the GPU released when Python is ter...,If your GPU memory isn’t freed even after Pyt...,PyTorch uses a caching memory allocator to sp...
97,"When Python is ended, why isn't the GPU relea...",If your GPU memory isn’t freed even after Pyt...,PyTorch uses a caching memory allocator to sp...
98,"When Python is ended, why isn't the GPU relea...",If your GPU memory isn’t freed even after Pyt...,PyTorch uses a caching memory allocator to sp...
99,Why isn't the GPU released when Python is fin...,If your GPU memory isn’t freed even after Pyt...,PyTorch uses a caching memory allocator to sp...


In [151]:
df_3 = train3.rename(columns={
    'X': 'question',
    'Y': 'answer',
    'Z': 'context'
})

df_3

,question,answer,context
0,In what platform do the key nn modules run?,FP32,torch.nn.intrinsic.qat This module implements ...
1,What module implements versions of the key nn ...,torch.nn.quantized,This module implements the functions you call ...
2,What is needed for quantization aware training?,fused operations,This module implements the versions of those f...
3,Conv2d and Linear() use rounding to simulate t...,INT8 quantization,This module implements the versions of those f...
4,What module implements versions of the key nn ...,torch.nn.quantized,This module implements the functions you call ...
...,...,...,...
18677,digamma Computes what derivative of the gamma ...,logarithmic derivative,digamma Computes the logarithmic derivative of...
18678,What computes the logarithmic derivative of th...,digamma,digamma Computes the logarithmic derivative of...
18679,exp Returns a new tensor with what of the elem...,exponential,exp Returns a new tensor with the exponential ...
18680,What is the Alias for torch.special.expm1?,expm1,expm1 Alias for torch.special.expm1().


In [153]:
final_pd = pd.concat([df_3, df_8, df_1, df_47, df_2, df_56])

In [154]:
final_pd.to_csv('data/final_pd.csv')